### Simple LangChain Demonstration
- As the goal of this demonstration, Our goal is to get suggestions for a Restaurant Name, and the menu items suitable for the name using any LLM of our choice.
- In this demo, let's use Gemini AI. 

#### Installation
- install gemini ai,langchain and langchain gemini module to use gemini ai
  - `py -m pip install google-generativeai langchain langchain-google-genai `


#### Connect with LLM
- use gemini ai to do a demo generation via langchain
  - create a gemini ai llm instance using the constructor ChatGoogleGenerativeAI


In [112]:
# import gemini api key from config
from utils import loadConfig, getConfig
loadConfig()
api_key = getConfig("gemini_apikey")

In [114]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
user_input = "What is the capital of France?"
messages = [HumanMessage(content=user_input)]
response = llm.invoke(messages)
print(response.content) 

Paris


#### Create prompts
- create a prompt which gets the input as a cuisine to generate a hotel name based on cuisine
  - In order to create a prompt in LangChain we need to use a prompt template, 
    - where we will have to provide the prompt with input templates 
    - along with the list of template input variables


In [23]:
from langchain.prompts import PromptTemplate

restaurant_name_prompt = PromptTemplate(
  input_variables=["cuisine"],
  template="Suggest one restaurant name for {cuisine} cuisine"
)

restaurant_name_prompt.format(cuisine =  "indian") #The formatted prompt 

'Suggest one restaurant name for indian cuisine'

- Why dont we just use a python formatted string instead?
  - Because we will have to pass the prompts via this prompt template to langchain chains to execute these prompts using our preferred LLMs.

- Similarly let us create a prompt to generate a menu based on hotel name

In [53]:
restaurant_menu_prompt = PromptTemplate(
  input_variables=["hotel"],
  template="For a hotel with the name {hotel}, suggest menu items with no categories, and comma separate them"
)

restaurant_menu_prompt.format(hotel =  "Masala Mantra") #The formatted prompt 

'For a hotel with the name Masala Mantra, suggest menu items with no categories, and comma separate them'

#### Chain prompts
- We will need to create a chain to execute this prompts with our LLM. So let's create two chains for the restaurant name and menu.
  - To create a chain we need 3 components
    - a prompt
    - a llm
    - and an output parser
      - We will use the built in stroutparser provided by LLM, this is a standard output parser, the derives the output in a string format from the LLM.

In [54]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
restaurant_name_chain = restaurant_name_prompt | llm | output_parser
restaurant_menu_chain = restaurant_menu_prompt | llm | output_parser

- Let's execute these chains independently with some sample inputs

In [58]:
print(restaurant_name_chain.invoke({"cuisine": "indian"}))
print(restaurant_menu_chain.invoke({"hotel": "Masala Mantra"}))

Tandoori Temptation
Paneer Makhani, Butter Chicken, Rogan Josh, Dal Makhani, Biryani, Naan, Tandoori Chicken, Gulab Jamun, Rasmalai, Mango Lassi


#### Run chains sequentially
- Now let's chain these prompts together


In [62]:
restaurant_name = restaurant_name_chain.invoke({"cuisine": "indian"})
restaurant_menu = restaurant_menu_chain.invoke({"hotel": restaurant_name})
print(restaurant_name)
print(restaurant_menu)

Aroma Spice
Tandoori Chicken, Butter Chicken, Paneer Tikka Masala, Biryani, Naan, Samosas, Pakoras, Lassi, Gulab Jamun, Kulfi


### LangChain and Agents Demo
- As we learned before on the intro on LangChain, we can create something called agents via LangChain
- Where the Agents use the reasoning ability of the LLM, to perform the next action using the tools in its toolkit.
- So using agents we can customize and personalize the usage to our use case, and develop internal tools and use external tools in combination to leverage the LLM's reasoning capabilities to serve our particular needs.

#### Let's create a sample LangChain agent
- Let's create an agent which looks uses the LLM to parse the input, and then uses the tools such as Wikipedia and Math tools to generate a response
- We can get list of tools available for langchain at langchain's website [list of tools available](https://python.langchain.com/docs/integrations/tools/)
- For this lets use tools from langchain community toolkit, so we need to additionally install langchain-community package, and other packages for wikipedia and math.


In [70]:
!py -m pip install -U langchain-community wikipedia numexpr


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 
#A ZERO_SHOT_REACT_DESCRIPTION is designed to perform on tasks without having been explicitly trained for them.
#They use the LLM to understand or generate a description for the input/query and based on this reacts to the description.
#Basically think->thought->react

#verbose = True -> prints out all the steps that the LLM goes through to provide the response

agent.run("When was Elon Musk born? What was his age in 1999?")





> Entering new AgentExecutor chain...
I should use Wikipedia to find out when Elon Musk was born, and then use Calculator to calculate his age in 1999.
Action: wikipedia
Action Input: when was elon musk born
Observation: Page: Musk family
Summary: The Musk family includes Maye Musk, a model and author; Errol Musk, a businessman and politician; their children Elon Musk, Kimbal Musk, and Tosca Musk, as well as nephew Lyndon Rive. Elon Musk's ex-wives are Justine Musk and Talulah Riley, and he has many children. Maye's father Joshua N. Haldeman was a notable chiropractor, aviator, and politician who was a well known proponent of Technocracy. Geographically, the family has roots in England, Canada, South Africa, and the United States of America.

Page: Acquisition of Twitter by Elon Musk
Summary: The business magnate Elon Musk initiated an acquisition of American social media company Twitter, Inc. on April 14, 2022, and concluded it on October 27, 2022. Musk had begun buying shares of th

'Elon Musk was born on June 28, 1971. He was 28 years old in 1999.'

- In the above reasoning steps, you can see how the wikipedia and math tools are being used along with the LLM's ability to understand and reason the input to generate a response.

### LangChain and memory Demo
- As we discussed during our intro on LangChain, LLMs do not have memory of their own, So if we want to create a context, we will have to pass the previous interactions as an input along with the current input to get an output that takes the context into consideration. Let's see an example how LLM's suffer from short term memory loss. All they can do is answer the current query.

In [79]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)

chain = PromptTemplate(template="Who won the cricket world cup in 1983?") | llm | output_parser
print(chain.invoke({}))

chain = PromptTemplate(template="Who was the captain?") | llm | output_parser
print(chain.invoke({}))


India
The context does not mention the name of the captain, so I cannot answer this question from the provided context.


- How to add memory to store the context?